## Shot Charts

In [39]:
import nba_api.stats.endpoints as nba
import mysql.connector
from mysql.connector import errorcode
import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


## Retrieve Shot Charts for Each Player in Dataset and Upload to Database

In [54]:
dbname = 'nba'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    db = dbname
)
cursor = cnx.cursor(buffered = True)

In [49]:
def get_shot_chart(team_id, player_id, date_from='', date_to='', context_measure_simple='FGA'):
    return nba.ShotChartDetail(team_id=team_id,player_id=player_id, date_from_nullable=date_from, date_to_nullable=date_to, context_measure_simple='FGA',timeout=60).get_data_frames()[0]

In [42]:
stmt = "SELECT TEAM_ID, PLAYER_ID FROM box_score"
cursor.execute(stmt)
team_player_combos = cursor.fetchall()

In [43]:
unique = list(set(team_player_combos))

In [44]:
stmt = "SELECT * from nba.shot_chart"
cursor.execute(stmt)
scores = cursor.fetchall()

field_names = [i[0] for i in cursor.description]

In [47]:
insert_statement = "INSERT INTO nba.shot_chart (GRID_TYPE, GAME_ID, GAME_EVENT_ID, PLAYER_ID, PLAYER_NAME, TEAM_ID, TEAM_NAME, PERIOD, MINUTES_REMAINING, SECONDS_REMAINING, EVENT_TYPE, ACTION_TYPE, SHOT_TYPE, SHOT_ZONE_BASIC, SHOT_ZONE_AREA, SHOT_ZONE_RANGE, SHOT_DISTANCE, LOC_X, LOC_Y, SHOT_ATTEMPTED_FLAG, SHOT_MADE_FLAG, GAME_DATE, HTM, VTM) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

In [51]:
count = 0
for combo in unique:
    df = get_shot_chart(combo[0], combo[1], date_from ='10-17-2017', date_to = '11-24-2019' )
    df_reordered = df[field_names]
    stats = [tuple(row) for row in df_reordered.values]
    cursor.executemany(insert_statement, stats)
    cnx.commit()
    count+=1
    print (count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [53]:
count = 0
for combo in unique[811:]:
    df = get_shot_chart(combo[0], combo[1], date_from ='10-17-2017', date_to = '11-24-2019' )
    df_reordered = df[field_names]
    stats = [tuple(row) for row in df_reordered.values]
    cursor.executemany(insert_statement, stats)
    cnx.commit()
    count+=1
    print (count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


## Explore Shot Charts for Preparation for Clustering

In [56]:
stmt = "SELECT * from nba.shot_chart"
cursor.execute(stmt)
scores = cursor.fetchall()

field_names = [i[0] for i in cursor.description]
shots = pd.DataFrame(scores)
shots.columns = field_names

In [58]:
shots.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM'],
      dtype='object')

In [69]:
shots.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021700885,643,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,1,31,...,Center(C),Less Than 8 ft.,2,-25,5,1,0,20180224,MIA,MEM
1,Shot Chart Detail,0021700885,645,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,1,27,...,Center(C),Less Than 8 ft.,0,0,-6,1,0,20180224,MIA,MEM
2,Shot Chart Detail,0021700898,657,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,1,19,...,Center(C),Less Than 8 ft.,2,17,20,1,1,20180226,BOS,MEM
3,Shot Chart Detail,0021700898,671,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,0,12,...,Center(C),Less Than 8 ft.,2,-18,10,1,1,20180226,BOS,MEM
4,Shot Chart Detail,0021700931,124,1627744,Brice Johnson,1610612763,Memphis Grizzlies,1,2,36,...,Center(C),8-16 ft.,15,23,151,1,0,20180302,MEM,DEN


In [61]:
shots.ACTION_TYPE.unique()

array(['Layup Shot', 'Tip Layup Shot', 'Driving Layup Shot',
       'Cutting Dunk Shot', 'Jump Shot', 'Cutting Layup Shot',
       'Hook Shot', 'Floating Jump shot', 'Putback Layup Shot',
       'Cutting Finger Roll Layup Shot', 'Dunk Shot',
       'Driving Floating Jump Shot', 'Driving Finger Roll Layup Shot',
       'Running Jump Shot', 'Turnaround Jump Shot', 'Running Layup Shot',
       'Driving Dunk Shot', 'Pullup Jump shot', 'Driving Hook Shot',
       'Reverse Layup Shot', 'Running Dunk Shot',
       'Turnaround Bank Hook Shot', 'Driving Reverse Layup Shot',
       'Turnaround Fadeaway shot', 'Fadeaway Jump Shot',
       'Running Finger Roll Layup Shot', 'Finger Roll Layup Shot',
       'Alley Oop Dunk Shot', 'Step Back Jump shot',
       'Running Pull-Up Jump Shot', 'Jump Bank Shot',
       'Alley Oop Layup shot', 'Driving Floating Bank Jump Shot',
       'Running Reverse Layup Shot', 'Turnaround Hook Shot',
       'Tip Dunk Shot', 'Putback Dunk Shot',
       'Turnaround Fadeaw

In [60]:
len(shots.ACTION_TYPE.unique())

46

In [64]:
shots[['LOC_X','LOC_Y', 'SHOT_DISTANCE']].head()

,LOC_X,LOC_Y,SHOT_DISTANCE
0,-25,5,2
1,0,-6,0
2,17,20,2
3,-18,10,2
4,23,151,15


In [70]:
shots17 = shots[shots.GAME_ID.str.contains('00217')]

In [72]:
simmons17 = shots17[shots['PLAYER_NAME']=='Ben Simmons']

/Users/marshall132/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [75]:

simmons17.groupby('ACTION_TYPE').agg('mean')

,GAME_EVENT_ID,PLAYER_ID,TEAM_ID,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG
ACTION_TYPE,,,,,,,,,,,
Alley Oop Dunk Shot,389.181818,1627732.0,1.610613e+09,2.727273,5.545455,24.727273,1.000000,-0.727273,11.727273,1.0,0.818182
Alley Oop Layup shot,313.666667,1627732.0,1.610613e+09,2.500000,8.000000,27.833333,1.666667,7.500000,13.500000,1.0,0.833333
Cutting Dunk Shot,397.705882,1627732.0,1.610613e+09,2.823529,4.941176,24.588235,0.882353,4.470588,8.588235,1.0,0.941176
Cutting Finger Roll Layup Shot,152.666667,1627732.0,1.610613e+09,1.666667,9.333333,18.000000,4.000000,-2.000000,40.333333,1.0,0.666667
Cutting Layup Shot,203.545455,1627732.0,1.610613e+09,1.818182,6.363636,26.090909,1.454545,10.090909,12.181818,1.0,0.909091
Driving Dunk Shot,330.777778,1627732.0,1.610613e+09,2.444444,5.311111,28.577778,0.555556,1.755556,7.711111,1.0,0.955556
Driving Finger Roll Layup Shot,264.138889,1627732.0,1.610613e+09,2.166667,6.611111,29.722222,2.861111,5.250000,27.111111,1.0,0.722222
Driving Floating Bank Jump Shot,331.307692,1627732.0,1.610613e+09,2.461538,4.692308,22.769231,6.615385,18.000000,48.153846,1.0,0.538462
Driving Floating Jump Shot,308.352941,1627732.0,1.610613e+09,2.441176,6.764706,32.941176,7.529412,4.852941,67.705882,1.0,0.323529


In [93]:
simmons_dumb = pd.get_dummies(simmons17, columns=['ACTION_TYPE'], prefix=[''], prefix_sep=[''])
simmons_dumb.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,Running Reverse Dunk Shot,Running Reverse Layup Shot,Step Back Bank Jump Shot,Step Back Jump shot,Tip Dunk Shot,Tip Layup Shot,Turnaround Bank Hook Shot,Turnaround Fadeaway shot,Turnaround Hook Shot,Turnaround Jump Shot
403975,Shot Chart Detail,0021700006,33,1627732,Ben Simmons,1610612755,Philadelphia 76ers,1,9,44,...,0,0,0,0,0,0,0,0,0,0
403976,Shot Chart Detail,0021700006,47,1627732,Ben Simmons,1610612755,Philadelphia 76ers,1,8,28,...,0,0,0,0,0,0,0,0,0,0
403977,Shot Chart Detail,0021700006,77,1627732,Ben Simmons,1610612755,Philadelphia 76ers,1,5,54,...,0,0,0,0,0,0,0,0,1,0
403978,Shot Chart Detail,0021700006,182,1627732,Ben Simmons,1610612755,Philadelphia 76ers,2,11,43,...,0,0,0,0,0,0,0,0,0,0
403979,Shot Chart Detail,0021700006,187,1627732,Ben Simmons,1610612755,Philadelphia 76ers,2,10,42,...,0,0,0,0,0,0,0,0,0,0


## Convert action_type columns into dummy variables so that I can apply aggregation functions to these to see the percentage of field goal attempts each player takes by shot type

In [102]:
shots17_dummy = pd.get_dummies(shots17, columns=['ACTION_TYPE','SHOT_TYPE'], prefix=['',''], prefix_sep=['',''])

In [103]:
shots17_dummy.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA',
       'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y',
       'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE', 'HTM', 'VTM',
       'Alley Oop Dunk Shot', 'Alley Oop Layup shot', 'Cutting Dunk Shot',
       'Cutting Finger Roll Layup Shot', 'Cutting Layup Shot',
       'Driving Bank Hook Shot', 'Driving Dunk Shot',
       'Driving Finger Roll Layup Shot', 'Driving Floating Bank Jump Shot',
       'Driving Floating Jump Shot', 'Driving Hook Shot', 'Driving Layup Shot',
       'Driving Reverse Dunk Shot', 'Driving Reverse Layup Shot', 'Dunk Shot',
       'Fadeaway Jump Shot', 'Finger Roll Layup Shot', 'Floating Jump shot',
       'Hook Bank Shot', 'Hook Shot', 'Jump Bank Shot', 'Jump Shot',
       'Layup Shot', 'Pullup Jump shot', 'Putback Dunk Shot',
       'Putb

In [107]:
keep_cols =['PLAYER_ID', 'PLAYER_NAME', 'SHOT_DISTANCE', 'SHOT_MADE_FLAG','Alley Oop Dunk Shot', 'Alley Oop Layup shot', 'Cutting Dunk Shot',
       'Cutting Finger Roll Layup Shot', 'Cutting Layup Shot',
       'Driving Bank Hook Shot', 'Driving Dunk Shot',
       'Driving Finger Roll Layup Shot', 'Driving Floating Bank Jump Shot',
       'Driving Floating Jump Shot', 'Driving Hook Shot', 'Driving Layup Shot',
       'Driving Reverse Dunk Shot', 'Driving Reverse Layup Shot', 'Dunk Shot',
       'Fadeaway Jump Shot', 'Finger Roll Layup Shot', 'Floating Jump shot',
       'Hook Bank Shot', 'Hook Shot', 'Jump Bank Shot', 'Jump Shot',
       'Layup Shot', 'Pullup Jump shot', 'Putback Dunk Shot',
       'Putback Layup Shot', 'Reverse Dunk Shot', 'Reverse Layup Shot',
       'Running Alley Oop Dunk Shot', 'Running Alley Oop Layup Shot',
       'Running Dunk Shot', 'Running Finger Roll Layup Shot',
       'Running Jump Shot', 'Running Layup Shot', 'Running Pull-Up Jump Shot',
       'Running Reverse Dunk Shot', 'Running Reverse Layup Shot',
       'Step Back Bank Jump Shot', 'Step Back Jump shot', 'Tip Dunk Shot',
       'Tip Layup Shot', 'Turnaround Bank Hook Shot',
       'Turnaround Fadeaway Bank Jump Shot', 'Turnaround Fadeaway shot',
       'Turnaround Hook Shot', 'Turnaround Jump Shot', '2PT Field Goal',
       '3PT Field Goal']

In [121]:
agg = shots17_dummy[keep_cols].groupby('PLAYER_NAME').agg('mean')

## Create a separate dataframe to calculate shot attempts per game to join with the above table

In [116]:
shots_per = shots17.groupby('PLAYER_NAME')['SHOT_ATTEMPTED_FLAG', 'GAME_ID'].agg(['sum',pd.Series.nunique])

In [118]:
shots_per.columns = ['total_shots', 'xx', 'xxx', 'games_played']
shots_per['attempts_per_game'] = shots_per['total_shots'] / shots_per['games_played']
shots_per.head()

,total_shots,xx,xxx,games_played,attempts_per_game
PLAYER_NAME,,,,,
Aaron Brooks,69,1,0021700052002170005200217000520021700056002170...,23,3.000000
Aaron Gordon,865,1,0021700005002170000500217000050021700005002170...,58,14.913793
Aaron Harrison,69,1,0021701076002170107600217010760021701076002170...,9,7.666667
Aaron Jackson,9,1,0021701230002170123000217012300021701230002170...,1,9.000000
Abdel Nader,149,1,0021700007002170000700217000070021700007002170...,38,3.921053


In [119]:
shots_per.sort_values(by='attempts_per_game', ascending=False)

,total_shots,xx,xxx,games_played,attempts_per_game
PLAYER_NAME,,,,,
Russell Westbrook,1687,1,0021700015002170001500217000150021700015002170...,80,21.087500
James Harden,1449,1,0021700002002170000200217000020021700002002170...,72,20.125000
Devin Booker,1054,1,0021700012002170001200217000120021700012002170...,54,19.518519
Anthony Davis,1462,1,0021700008002170000800217000080021700008002170...,75,19.493333
Damian Lillard,1415,1,0021700012002170001200217000120021700012002170...,73,19.383562
...,...,...,...,...,...
Erik McCree,2,1,00217007930021701013,2,1.000000
Edmond Sumner,1,1,0021701213,1,1.000000
DaQuan Jeffries,1,1,0021900217,1,1.000000


In [124]:
merged = pd.merge(left=agg, right=shots_per['attempts_per_game'], left_on=agg.index, right_on=shots_per.index)